In [1]:
import csv
import math
import numpy as np
import pandas as pd
import scipy 
import os
import random
    
#class Naive_Bayes:
    
def makePrediction(pPredict, nPredict):
    prediction = []
    for i in range(len(pPredict)):
        if pPredict[i]>nPredict[i]:
            prediction.append(1)
        else:
            prediction.append(0)
    return prediction

def getAccuracy(prediction, testC):
    trueP, trueN, falseP, falseN = 0,0,0,0

    for i in range(len(prediction)):
        if prediction[i] == testC[i]:
            if prediction[i] == [1]:
                trueP += 1
            else:
                trueN += 1
        else:
            if prediction[i] == 1:
                falseP += 1
            else:
                falseN += 1
    accuracy = ((trueP + trueN)/(len(prediction)))*100
    precision = trueP/float(trueP + falseN)
    recall = trueP/float(trueP + falseN)

    return [accuracy, precision, recall]



def calculate (meanAndVariance, data, prior):
    #convert df to list
    dataList = data.values.tolist()
    predict = []
    for i in range(len(dataList)):
        p = 1
        for j in range(len(dataList[i])):
               exponent = math.exp(-(math.pow(dataList[i][j]-meanAndVariance[j][0],2)/(2*math.pow(meanAndVariance[j][1],2))))
               p = p*(1/(math.sqrt(2*math.pi) *meanAndVariance[j][1])) * exponent
               predict.append((p*prior))
    return predict


def variance (data, MV):
    dataList = data.values.tolist()

    for i in range(8):
        d = 0
        
        for j in range(len(dataList)):
            d += pow((dataList[j][i] - MV [i][0]),2)
        d = (d/float((len(data['passing_grade']))))
        MV[i].append(math.sqrt(d))
    return MV

def mean (dataP, dataN):
    positiveMV = []
    negativeMV = []

    for column in dataP:
        #break loop since you don't need to calc mean
        if column == 'passing_grade':
            break
        meanVectorP = [sum(dataP[column])/(len(dataP[column]))]
        positiveMV.append(meanVectorP)
        meanVectorN = [sum(dataN[column])/(len(dataN[column]))]
        negativeMV.append(meanVectorN)
    return [positiveMV, negativeMV]


def splitByPassingGrade (data):
    #split passing
    positiveT = data[data['passing_grade']==1]
    #split failing
    negativeT = data[data['passing_grade']==0]

    return[positiveT, negativeT]


def splitData(data):
    #split the data 20% test, 80 % train
    testRow = math.floor((len(data)*20)/100)

    test = data.iloc[:testRow,:]
    train = data.iloc[testRow:,:]
    return [test, train]




In [2]:
df1 = pd.read_csv(open('NB_class.py/data.csv/student-mat.csv'))
df2 = pd.read_csv(open('NB_class.py/data.csv/student-por.csv'))
df = df1.append(df2)
print(df.head())
df.shape

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]


(1044, 33)

In [3]:
df = df.drop(columns = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher','internet', 'romantic'])
df.rename(columns={'G1':'period1_score'}, inplace = True)
df.rename(columns={'G2': 'period2_score'}, inplace = True)
df.rename(columns={'G3': 'final_grade'}, inplace = True)
df.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,period1_score,period2_score,final_grade
0,18,4,4,2,2,0,4,3,4,1,1,3,6,5,6,6
1,17,1,1,1,2,0,5,3,3,1,1,3,4,5,5,6
2,15,1,1,1,2,3,4,3,2,2,3,3,10,7,8,10
3,15,4,2,1,3,0,3,2,2,1,1,5,2,15,14,15
4,16,3,3,1,2,0,4,3,2,1,2,5,4,6,10,10


In [12]:
df["passing_grade"] = 0*len(df['final_grade'])
print(df.head())
df.shape

   age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  goout  \
0   18     4     4           2          2         0       4         3      4   
1   17     1     1           1          2         0       5         3      3   
2   15     1     1           1          2         3       4         3      2   
3   15     4     2           1          3         0       3         2      2   
4   16     3     3           1          2         0       4         3      2   

   Dalc  Walc  health  absences  period1_score  period2_score  final_grade  \
0     1     1       3         6              5              6            6   
1     1     1       3         4              5              5            6   
2     2     3       3        10              7              8           10   
3     1     1       5         2             15             14           15   
4     1     2       5         4              6             10           10   

   passing_grade  
0              0  
1           

(1044, 17)

In [7]:
for i in len(df["passing_grade"]):
    if i.df['final_grade'] in range(14,20):
        i.df['passing_grade'] = 1
    else:
        i.df['passing_grade']= 0
df.head()

TypeError: 'int' object is not iterable

In [5]:

passing_grade = range(14,20)
#randomize data
df = df.iloc[np.random.permutation(len(df))]
test, train = splitData(df)
print("number of test rows", len(test['passing_grade']))
print("number of training rows", len(train['passing_grade']))

number of test rows 208
number of training rows 836


In [6]:
#get initial probability
positiveT, negativeT = splitByPassingGrade(train)

priorP = len(positiveT['passing_grade'])/len(train['passing_grade'])
priorN = len(negativeT['passing_grade'])/len(train['passing_grade'])

print('Prior prob of not passing', priorN)
print('prior prob of passing', priorP)

Prior prob of not passing 0.05143540669856459
prior prob of passing 0.0011961722488038277


In [28]:
#get mean and variance
positiveMV, negativeMV = mean(positiveT, negativeT)

positiveMV = variance(positiveT, positiveMV)
negativeMB = variance(negativeT, negativeMV)

testF = test.iloc[:,0:8]
testC = test.iloc[:,-1]

In [29]:
#find predictions of both classes
pPredict = calculate(positiveMV, testF, priorP)
nPredict = calculate(negativeMV, testF,priorN)

#calculate prediction
prediction = makePrediction(pPredict, nPredict)
print('Prediction Result:', prediction)

ZeroDivisionError: float division by zero

In [31]:
accuracy, precision, recall, = getAccuracy(prediction, testC.valuse.tolist())
print('accuracy', accuracy,'%\nprecision:', precision, '%\nrecall:', recall)


0.0011961722488038277
